# Setup

## Imports

In [4]:
# Import importlib to reload modules and sys and os to add the path for other imports
import importlib
import sys
import os

# Append the parent directory to the path to import the necessary modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the utilities and the dataloader
from utils import selfutil, parseutil
from classes import SpreadsheetDataLoader

# Now reload the modules to ensure they are up-to-date
importlib.reload(selfutil)
importlib.reload(parseutil)
importlib.reload(SpreadsheetDataLoader)

# Import the funcs needed from utils
from utils.selfutil import get_vocab2, create_embeddings
from utils.parseutil import process_spreadsheet

# Import the SpreadsheetDataLoader class
from classes.SpreadsheetDataLoader import SpreadsheetDataLoader

# Other regular imports
import torch.nn as nn
import torch
from tqdm import tqdm
import gc
import os
import pandas as pd
import math
import time
import numpy as np
import random

# Setup device as a global constant
# Define the desired device string
devstr = "mps"  # Change to "cpu" or "mps" as needed
gpu = False if (devstr == 'cpu') else True

# Determine the device to use
if devstr == 'cpu':
    DEVICE = 'cpu'
elif devstr.startswith('cuda') and torch.cuda.is_available():
    DEVICE = torch.device(devstr)
elif devstr == 'mps' and torch.backends.mps.is_available():
    DEVICE = torch.device(devstr)
else:
    DEVICE = 'cpu'  # Default to CPU if none of the above are available

print(DEVICE)

mps


## Pre-Vocab

In [5]:
# Set the directory containing the spreadsheets
data_dir = '../data/train_big/'

# Get the vocabulary object and embeddings
spreadsheet_vocab = get_vocab2(data_dir)
spreadsheet_wvs = create_embeddings(spreadsheet_vocab)

print(f'Word Embeddings Shape: {spreadsheet_wvs.shape}')

Processing Files: 100%|██████████| 799/799 [00:52<00:00, 15.12it/s]


799(P) = 799(G) + 0(E)
Unique Tokens: 140364
Vocab Size: 50000


100%|██████████| 50000/50000 [00:00<00:00, 130897.01it/s]

Word Embeddings Shape: torch.Size([50000, 50])


# Process Spreadsheet Checker


In [6]:
import os
from tqdm import tqdm  # For progress bar

# Define the directory containing the files
data_dir = '../data/val_big/'

# Extract all .xls, .xlsx, and .csv file paths into a list
spreadsheet_files = [
    os.path.join(data_dir, file_name)
    for file_name in os.listdir(data_dir)
    if file_name.endswith(('.xls', '.xlsx', '.csv'))
]

# Initialize a list to track files that failed processing or returned None
failed_files = []
none_returned_files = []

# Iterate through the list of spreadsheet files
for file_path in tqdm(spreadsheet_files, desc="Processing spreadsheets"):
    try:
        # Attempt to process the file using the process_spreadsheet function
        result = process_spreadsheet(file_path, spreadsheet_vocab, 100, 100, 32)
        
        # Check if the function returned None
        if result is None:
            print(f"Function returned None for: {file_path}")
            none_returned_files.append(file_path)
        else:
            # You can handle the result further if needed
            x_tok, y_tok = result

    except Exception as e:
        # Log the file name and the error
        print(f"Failed to process: {file_path}")
        print(f"Error: {e}")
        failed_files.append(file_path)

# Print summary of failed files
print("\nProcessing completed.")
if failed_files:
    print("The following files failed processing:")
    for failed_file in failed_files:
        print(failed_file)
else:
    print("All files processed successfully.")

# Print summary of files where function returned None
if none_returned_files:
    print("\nThe following files returned None:")
    for none_file in none_returned_files:
        print(none_file)
else:
    print("\nNo files returned None.")


Processing spreadsheets:  62%|██████▏   | 62/100 [00:23<00:15,  2.38it/s]

Function returned None for: ../data/val_big/sara_shackleton_001_1_1.pst.346.xls


Processing spreadsheets: 100%|██████████| 100/100 [00:43<00:00,  2.32it/s]


Processing completed.
All files processed successfully.

The following files returned None:
../data/val_big/sara_shackleton_001_1_1.pst.346.xls


## Temporary tokenize_pad Func

Helper function to remove the encoding aspect for simplicity

In [6]:
def tokenize_pad2( cell_value, vocab, pad_length = 32 ):
    """
    Tokenizes, pads, and encodes a cell value.

    Args:
        cell_value (str): The string value of the cell to be processed.
        vocab: Vocabulary object for encoding tokens.
        pad_length (int, optional): The length to which the tokenized list is padded. Defaults to 32.

    Returns:
        List[int]: A list of encoded tokens, padded or truncated to the specified length.
    """
    # Convert the cell value to lowercase and tokenize
    tokens = tokenize(cell_value.lower())

    # Add the <cls> and <eos> tokens
    input_tokens = tokens

    # Pad or truncate the token list to the specified pad_length
    # if len(input_tokens) < pad_length:
    #     remaining_length = pad_length - len(input_tokens)
    #     input_tokens.extend([ '<pad>' ] * remaining_length)
    # else:
    #     input_tokens = input_tokens[ :pad_length ]

    # Encode the tokens using the provided vocabulary
    #toks_encoded = [ vocab.encode(tok) for tok in input_tokens ]
    toks_encoded = [ tok for tok in input_tokens ]

    return toks_encoded

## XLS Files 

### Original process_xls function

In [7]:
def process_xls( file_path, vocab, max_rows, max_cols, pad_length ):
    """
    Extracts metadata for each cell in the first sheet of the given spreadsheet and tokenizes the cell values.

    Args:
        file_path (str): The path to the spreadsheet file.
        vocab (dict): A dictionary mapping tokens to their indices for encoding.
        max_rows (int): The maximum number of rows to process.
        max_cols (int): The maximum number of columns to process.
        pad_length (int): The length to which the tokens should be padded.

    Returns:
        x_tok (torch.Tensor): A 3D tensor of size (max_rows, max_cols, pad_length) containing tokenized cell values.
        y_tok (torch.Tensor): A 3D tensor of size (max_rows, max_cols, 7) containing metadata for each cell:
            - Index 0: Cell type (int)
            - Index 1: Fill (int)
            - Index 2: Alignment (int)
            - Index 3: Font family (int)
            - Index 4: Style (int)
            - Index 5: Borders (int)
            - Index 6: Merge status (int)
    """

    def get_fill( xf_record, workbook ):
        """
        Determines whether the cell is filled with a background color.

        Args:
            xf_record (xlrd.formatting.XFRecord): The XFRecord object containing formatting information for the cell.
            workbook (xlrd.Book): The workbook object containing the cell.

        Returns:
            int: 1 if the cell is filled with a color other than white, 0 otherwise.
        """

        # Retrieve the color of the cell's background using the pattern_colour_index
        bg_color_rgb = workbook.colour_map.get(xf_record.background.pattern_colour_index)

        # Return 0 if the cell is not filled or filled with white; otherwise, return 1
        return 0 if bg_color_rgb in [ None, (255, 255, 255) ] else 1

    def get_merge( row, col, sheet ):
        """
        Determines the merge status of a cell in terms of its vertical and horizontal position within a merged range.

        Int to Merge Status Mapping:
            1 -> Outside any merged range ('O')
            2 -> Beginning of a merged range ('B')
            3 -> Inside a merged range but not at the beginning ('I')

        Args:
            row (int): The row index of the cell.
            col (int): The column index of the cell.
            sheet (xlrd.sheet.Sheet): The sheet object containing the merged cells information.

        Returns:
            int: A 2-digit integer where the first digit represents the vertical position
                 and the second digit represents the horizontal position within a merged range.
        """

        # Initialize tags as 0 (outside) by default
        vertical_tag = 0
        horizontal_tag = 0

        # Iterate over the merged cells in the sheet
        for start_row, end_row, start_col, end_col in sheet.merged_cells:

            # Check if the current cell is within a merged range
            if start_row <= row < end_row and start_col <= col < end_col:

                # Determine vertical tag
                if row == start_row:
                    vertical_tag = 1  # Beginning of vertical range
                else:
                    vertical_tag = 2  # Inside vertical range

                # Determine horizontal tag
                if col == start_col:
                    horizontal_tag = 1  # Beginning of horizontal range
                else:
                    horizontal_tag = 2  # Inside horizontal range

                # Break the loop as the cell is part of a merged range
                break

        # Return the combined vertical and horizontal tags
        return horizontal_tag, vertical_tag

    def get_data( cell, workbook, df, row, col ):
        """
        Determines the type of the cell, handles necessary conversions, and retrieves the cell value.

        Args:
            cell (xlrd.sheet.Cell): The cell object to determine the type and retrieve the value.
            workbook (xlrd.Book): The workbook object containing the cell.
            df (pandas.DataFrame): DataFrame representation of the sheet for value extraction.
            row (int): The row index of the cell.
            col (int): The column index of the cell.

        Returns:
            tuple: A tuple containing the determined type of the cell (int) and the cell value (str).
        """
        xlrd_error_map = {
            0: "#NULL!", 7: "#DIV/0!", 15: "#VALUE!", 23: "#REF!", 29: "#NAME?", 36: "#NUM!", 42: "#N/A", 43: "#GETTING_DATA",
        }

        # Empty cell with no formatting
        if cell.ctype == 0:
            return 0, ''

        # If text cell then check further to get exact text type
        elif cell.ctype == 1:
            format_str = workbook.format_map[ workbook.xf_list[ cell.xf_index ].format_key ].format_str

            if '#,##0' in format_str:
                return 6, str(cell.value)  # Currency type

            return 1, str(cell.value)  # Text type

        # If numeric cell then check further to get exact numeric type
        elif cell.ctype == 2:
            format_str = workbook.format_map[ workbook.xf_list[ cell.xf_index ].format_key ].format_str

            if format_str == '0.00E+00':
                return 7, f"{cell.value:.2e}"  # Scientific subclass of number

            elif '#,##0' in format_str:

                if '$€' in format_str:
                    return 6, '€' + str(float(cell.value))

                return 6, '$' + str(float(cell.value))


            elif '%' in format_str:
                return 5, str(cell.value * 100) + '%'  # Percentage type

            elif isinstance(cell.value, float):
                if cell.value.is_integer():
                    return 3, str(df.iat[ row, col ])  # Integer type
                return 4, str(df.iat[ row, col ])  # Float type

            return 2, str(df.iat[ row, col ])  # General numeric type

        # If date cell then check further to get exact date type
        elif cell.ctype == 3:
            try:
                date_tuple = xldate_as_tuple(cell.value, workbook.datemode)

                if date_tuple[ :3 ] == (0, 0, 0):
                    return 9, str(df.iat[ row, col ])  # Time type

                elif date_tuple[ 3: ] == (0, 0, 0):
                    return 8, str(df.iat[ row, col ])  # Date type

                return 10, str(df.iat[ row, col ])  # Datetime type

            except (XLDateError, XLDateAmbiguous):
                return 10, str(df.iat[ row, col ])  # Datetime type

        # Boolean cell
        elif cell.ctype == 4:
            return 11, str(df.iat[ row, col ])

        # Error cell
        elif cell.ctype == 5:
            return 12, xlrd_error_map.get(cell.value, f"#ERR{cell.value}")

        # Blank cell (Empty cell with formatting)
        elif cell.ctype == 6:
            return 13, ''

        # Unknown type not in our keys return by default
        return 14, ''

    # Initialize 3D tensor of size row x col x seq = 100x100x32
    x_tok = torch.zeros((max_rows, max_cols, pad_length), dtype = torch.long)

    # Initialize y_tok tensor to store metadata for each cell of size row x col x 17
    y_tok = torch.zeros((max_rows, max_cols, 17), dtype = torch.long)

    # Open the .xls file with formatting_info=True to get the metadata
    workbook = xlrd.open_workbook(filename = file_path, formatting_info = True)

    # Access the first sheet in the workbook
    sheet = workbook.sheet_by_index(0)

    # Get the df for some parsing
    df_read = pd.read_excel(file_path, header = None, dtype = str, na_values = ' ', keep_default_na = False, engine = 'xlrd')

    # Convert to the size of df required
    df = df_read.reindex(index = range(max_rows), columns = range(max_cols), fill_value = '')

    # Initialize checking df DataFrame to store cell values for parse test
    check_df = df_read.reindex(index = range(max_rows), columns = range(max_cols), fill_value = '')
    
    # Iterate over each combination of rows and columns in the sheet, up to the 100th row and column
    for row, col in itertools.product(range(min(sheet.nrows, max_rows)), range(min(sheet.ncols, max_cols))):
    
        # Retrieve the cell object from the sheet
        cell = sheet.cell(row, col)
    
        # Access the XFRecord object using the cell's XF index
        xf_record = workbook.xf_list[ cell.xf_index ]
    
        # Get cell train
        cell_type, cell_value = get_data(cell, workbook, df, row, col)
    
        # Set the tokens for current cell in x_tok
        check_df.iloc[row, col] = cell_value
        x_tok[ row, col, : ] = torch.tensor(tokenize_pad(cell_value, vocab, pad_length), dtype = torch.long)
    
        # Determine the fill of the cell
        y_tok[ row, col, 0 ] = cell_type
        y_tok[ row, col, 1 ] = get_fill(xf_record, workbook)  # Store 'fill'
        y_tok[ row, col, 2 ] = xf_record.alignment.hor_align  # Store 'halign'
        y_tok[ row, col, 3 ] = xf_record.alignment.vert_align  # Store 'valign'
        y_tok[ row, col, 4 ] = workbook.font_list[ xf_record.font_index ].family  # Store 'font family'
        y_tok[ row, col, 5 ] = workbook.font_list[ xf_record.font_index ].height // 20  # Store 'font size'
        y_tok[ row, col, 6 ] = workbook.font_list[ xf_record.font_index ].bold  # Store 'bold'
        y_tok[ row, col, 7 ] = workbook.font_list[ xf_record.font_index ].italic  # Store 'italic'
        y_tok[ row, col, 8 ] = workbook.font_list[ xf_record.font_index ].underline_type  # Store 'underline'
        y_tok[ row, col, 9 ] = workbook.font_list[ xf_record.font_index ].escapement  # Store 'escapement'
        y_tok[ row, col, 10 ] = 1 if (rgb := workbook.colour_map.get(workbook.font_list[ xf_record.font_index ].colour_index)) and rgb != (
        0, 0, 0) else 0  # Store 'colour'
        y_tok[ row, col, 11 ] = xf_record.border.top_line_style  # Store top border style
        y_tok[ row, col, 12 ] = xf_record.border.bottom_line_style  #
        y_tok[ row, col, 13 ] = xf_record.border.left_line_style  # Store 'left border style'
        y_tok[ row, col, 14 ] = xf_record.border.right_line_style  # Store 'right border style'
        y_tok[ row, col, 15 ], y_tok[ row, col, 16 ] = get_merge(row, col, sheet)  # Store 'hmerge' and 'vmerge'
    
        # Check if empty cell or blank cell with no metadata
        if y_tok[ row, col, 0 ] == 0 or (y_tok[ row, col, 0 ] == 13 and y_tok[ row, col, 3 ] == 2 and all(
            y_tok[ row, col, pos ] == 0 for pos in [ i for i in range(1, 17) if i not in [ 3, 4, 5 ] ])):
            # Set to 0 tensor
            y_tok[ row, col, : ] = torch.zeros(17, dtype = torch.long)
    
    # Return the 100x100 2D NumPy array containing the metadata for each cell
    return check_df, y_tok

In [8]:
# Define tester file
test_file = '../data/train_big/parse_input.xls'

res_df, _ = process_xls(test_file, spreadsheet_vocab, 100, 100, 32)
display(res_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,,Monday,Tuesday,Wednesday,Thursday,friday,saturday,Sunday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,Val1,test,2024-09-08 00:00:00,1pm,Boolean,dh,Some,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2,20,40,80,160,320,640,1280,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,3,20,40,80,36.285031392744614,106.62767516629195,This is a multiple word sentence,2036.3150544362386,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,4,20,40,80,42.22922635061216,106.10423396091338,Test,TestCool,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,5,20,40,80,42.03629416295762,108.63591631231957,Test2,Test2233.720799713594,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,6,20,1.23e-09,2.46e-09,133.91797458573245,209.71724867157806,Test3,Test3242.995896871674,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,7,20,1.23e+02,246,138.00787964443867,217.63445971991325,Test,Test253.5164831058,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,8,20,1.23e+02,246,144.4672835613178,226.9020562300975,Test,Test265.436777227406,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,9,20,40,80,145.23283271104114,230.17938243850983,1999-01-02 00:00:00,36162279.069807278528,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### NEW test_xls function

In [7]:
def process_xls( file_path, vocab, max_rows, max_cols, pad_length ):
	"""
    Extracts metadata for each cell in the first sheet of the given spreadsheet and tokenizes the cell values.

    Args:
        file_path (str): The path to the spreadsheet file.
        vocab (dict): A dictionary mapping tokens to their indices for encoding.
        max_rows (int): The maximum number of rows to process.
        max_cols (int): The maximum number of columns to process.
        pad_length (int): The length to which the tokens should be padded.

    Returns:
        x_tok (torch.Tensor): A 3D tensor of size (max_rows, max_cols, pad_length) containing tokenized cell values.
        y_tok (torch.Tensor): A 3D tensor of size (max_rows, max_cols, 7) containing metadata for each cell:
            - Index 0: Cell type (int)
            - Index 1: Fill (int)
            - Index 2: Alignment (int)
            - Index 3: Font family (int)
            - Index 4: Style (int)
            - Index 5: Borders (int)
            - Index 6: Merge status (int)
    """

	def get_fill( xf_record, workbook ):
		"""
        Determines whether the cell is filled with a background color.

        Args:
            xf_record (xlrd.formatting.XFRecord): The XFRecord object containing formatting information for the cell.
            workbook (xlrd.Book): The workbook object containing the cell.

        Returns:
            int: 1 if the cell is filled with a color other than white, 0 otherwise.
        """

		# Retrieve the color of the cell's background using the pattern_colour_index
		bg_color_rgb = workbook.colour_map.get(xf_record.background.pattern_colour_index)

		# Return 0 if the cell is not filled or filled with white; otherwise, return 1
		return 0 if bg_color_rgb in [ None, (255, 255, 255) ] else 1

	def get_merge( row, col, sheet ):
		"""
        Determines the merge status of a cell in terms of its vertical and horizontal position within a merged range.

        Int to Merge Status Mapping:
            1 -> Outside any merged range ('O')
            2 -> Beginning of a merged range ('B')
            3 -> Inside a merged range but not at the beginning ('I')

        Args:
            row (int): The row index of the cell.
            col (int): The column index of the cell.
            sheet (xlrd.sheet.Sheet): The sheet object containing the merged cells information.

        Returns:
            int: A 2-digit integer where the first digit represents the vertical position
                 and the second digit represents the horizontal position within a merged range.
        """

		# Initialize tags as 0 (outside) by default
		vertical_tag = 0
		horizontal_tag = 0

		# Iterate over the merged cells in the sheet
		for start_row, end_row, start_col, end_col in sheet.merged_cells:

			# Check if the current cell is within a merged range
			if start_row <= row < end_row and start_col <= col < end_col:

				# Determine vertical tag
				if row == start_row:
					vertical_tag = 1  # Beginning of vertical range
				else:
					vertical_tag = 2  # Inside vertical range

				# Determine horizontal tag
				if col == start_col:
					horizontal_tag = 1  # Beginning of horizontal range
				else:
					horizontal_tag = 2  # Inside horizontal range

				# Break the loop as the cell is part of a merged range
				break

		# Return the combined vertical and horizontal tags
		return horizontal_tag, vertical_tag

	def get_data( cell, workbook, df, row, col ):
		"""
        Determines the type of the cell, handles necessary conversions, and retrieves the cell value.

        Args:
            cell (xlrd.sheet.Cell): The cell object to determine the type and retrieve the value.
            workbook (xlrd.Book): The workbook object containing the cell.
            df (pandas.DataFrame): DataFrame representation of the sheet for value extraction.
            row (int): The row index of the cell.
            col (int): The column index of the cell.

        Returns:
            tuple: A tuple containing the determined type of the cell (int) and the cell value (str).
        """
		xlrd_error_map = {
			0: "#NULL!", 7: "#DIV/0!", 15: "#VALUE!", 23: "#REF!", 29: "#NAME?", 36: "#NUM!", 42: "#N/A", 43: "#GETTING_DATA",
		}

		# Empty cell with no formatting
		if cell.ctype == 0:
			return 0, ''

		# If text cell then check further to get exact text type
		elif cell.ctype == 1:
			format_str = workbook.format_map[ workbook.xf_list[ cell.xf_index ].format_key ].format_str

			if '#,##0' in format_str:
				return 6, str(cell.value)  # Currency type

			if format_str == '0.00E+00':
				return 7, f"{cell.value}"  # Scientific subclass of text

			return 1, str(cell.value)  # Text type

		# If numeric cell then check further to get exact numeric type
		elif cell.ctype == 2:
			format_str = workbook.format_map[ workbook.xf_list[ cell.xf_index ].format_key ].format_str

			if format_str == '0.00E+00':
				return 7, f"{cell.value:.2e}"  # Scientific subclass of number

			elif '#,##0' in format_str:

				if '$€' in format_str:
					return 6, '€' + str(float(cell.value))

				return 6, '$' + str(float(cell.value))


			elif '%' in format_str:
				return 5, str(cell.value * 100) + '%'  # Percentage type

			elif isinstance(cell.value, float):
				if cell.value.is_integer():
					return 3, str(df.iat[ row, col ])  # Integer type
				return 4, str(df.iat[ row, col ])  # Float type

			return 2, str(df.iat[ row, col ])  # General numeric type

		# If date cell then check further to get exact date type
		elif cell.ctype == 3:
			try:
				date_tuple = xldate_as_tuple(cell.value, workbook.datemode)

				if date_tuple[ :3 ] == (0, 0, 0):
					return 9, str(df.iat[ row, col ])  # Time type

				elif date_tuple[ 3: ] == (0, 0, 0):
					return 8, str(df.iat[ row, col ])  # Date type

				return 10, str(df.iat[ row, col ])  # Datetime type

			except (XLDateError, XLDateAmbiguous):
				return 10, str(df.iat[ row, col ])  # Datetime type

		# Boolean cell
		elif cell.ctype == 4:
			return 11, str(df.iat[ row, col ])

		# Error cell
		elif cell.ctype == 5:
			return 12, xlrd_error_map.get(cell.value, f"#ERR{cell.value}")

		# Blank cell (Empty cell with formatting)
		elif cell.ctype == 6:
			return 13, ''

		# Unknown type not in our keys return by default
		return 14, ''

	# Initialize 3D tensor of size row x col x seq = 100x100x32
	x_tok = torch.zeros((max_rows, max_cols, pad_length), dtype = torch.long)

	# Initialize y_tok tensor to store metadata for each cell of size row x col x 17
	y_tok = torch.zeros((max_rows, max_cols, 17), dtype = torch.long)

	# Open the .xls file with formatting_info=True to get the metadata
	workbook = xlrd.open_workbook(filename = file_path, formatting_info = True)

	# Apply the BIFF check to check for BIFF version
	if workbook.biff_version != 80:
		raise ValueError(f"{workbook.biff_version}: {file_path}")

	# Access the first sheet in the workbook
	sheet = workbook.sheet_by_index(0)

	# Get the df for some parsing
	df_read = pd.read_excel(file_path, header = None, dtype = str, na_values = ' ', keep_default_na = False, engine = 'xlrd')

	# Convert to the size of df required
	df = df_read.reindex(index = range(max_rows), columns = range(max_cols), fill_value = '')

	# Initialize checking df DataFrame to store cell values for parse test
	#check_df = df_read.reindex(index = range(max_rows), columns = range(max_cols), fill_value = '')

	# Iterate over each combination of rows and columns in the sheet, up to the 100th row and column
	for row, col in itertools.product(range(min(sheet.nrows, max_rows)), range(min(sheet.ncols, max_cols))):

		# Retrieve the cell object from the sheet
		cell = sheet.cell(row, col)

		# Access the XFRecord object using the cell's XF index
		xf_record = workbook.xf_list[ cell.xf_index ]

		# Get cell train
		cell_type, cell_value = get_data(cell, workbook, df, row, col)

		# Set the tokens for current cell in x_tok
		#check_df.iloc[row, col] = cell_value
		x_tok[ row, col, : ] = torch.tensor(tokenize_pad(cell_value, vocab, pad_length), dtype = torch.long)

		# Set y_tok for current cell in y_tok
		y_tok[ row, col, 0 ] = cell_type
		y_tok[ row, col, 1 ] = get_fill(xf_record, workbook)  # Store 'fill'
		y_tok[ row, col, 2 ] = xf_record.alignment.hor_align  # Store 'halign'
		y_tok[ row, col, 3 ] = xf_record.alignment.vert_align  # Store 'valign'
		y_tok[ row, col, 4 ] = workbook.font_list[ xf_record.font_index ].family  # Store 'font family'
		y_tok[ row, col, 5 ] = workbook.font_list[ xf_record.font_index ].height // 20  # Store 'font size'
		y_tok[ row, col, 6 ] = workbook.font_list[ xf_record.font_index ].bold  # Store 'bold'
		y_tok[ row, col, 7 ] = workbook.font_list[ xf_record.font_index ].italic  # Store 'italic'
		y_tok[ row, col, 8 ] = workbook.font_list[ xf_record.font_index ].underline_type  # Store 'underline'
		y_tok[ row, col, 9 ] = workbook.font_list[ xf_record.font_index ].escapement  # Store 'escapement'
		y_tok[ row, col, 10 ] = 1 if (rgb := workbook.colour_map.get(workbook.font_list[ xf_record.font_index ].colour_index)) and rgb != (
			0, 0, 0) else 0  # Store 'colour'
		y_tok[ row, col, 11 ] = xf_record.border.top_line_style  # Store top border style
		y_tok[ row, col, 12 ] = xf_record.border.bottom_line_style  #
		y_tok[ row, col, 13 ] = xf_record.border.left_line_style  # Store 'left border style'
		y_tok[ row, col, 14 ] = xf_record.border.right_line_style  # Store 'right border style'
		y_tok[ row, col, 15 ], y_tok[ row, col, 16 ] = get_merge(row, col, sheet)  # Store 'hmerge' and 'vmerge'

		# If it is an empty/blank cell then further processing is required
		if y_tok[ row, col, 0 ] in [ 0, 13, 14 ]:

			# Set font-related metadata to 0 (indices 4 to 10)
			y_tok[ row, col, 4:11 ] = 0

			# If the cell is further of the type center across cells
			if y_tok[ row, col, 2 ] == 6:

				# Define variable to store col of where this center continuous is starting
				start_col = col

				# Loop until we find a cell that is centered across cells but not empty or blank
				while y_tok[ row, start_col, 2 ] == 6 and (y_tok[ row, start_col, 0 ] == (0 or 13)):

					# Move to the left
					start_col = start_col - 1

				# Set the x_tok and y_tok tensor to these values
				#check_df.iloc[row, col] = check_df.iloc[row, start_col]
				x_tok[ row, col, : ] = x_tok[ row, start_col, : ]
				y_tok[ row, col, : ] = y_tok[ row, start_col, : ]

				# print(f'Cell: ({row}, {col})')  # print(f'Value: {check_df.iloc[row, col]}')  # print(f'Type: {cell_type}')  # print(f"x_tok:\n{x_tok[row, col, :]}\n")  # print(f"y_tok:\n{y_tok[row, col, :]}\n")

	# Iterate over the merged cells in the sheet
	for start_row, end_row, start_col, end_col in sheet.merged_cells:

		# # Print the merged cell locations for debugging
		# print(f'\n################NEW RANGE####################')
		# print(f'\nInitial Start Row: {start_row}, End Row: {end_row}, Start Col: {start_col}, End Col: {end_col},')

		# Derive adjusted indices for accessing the combined cell
		adj_sr = start_row
		adj_er = end_row - 1  # Cause end index is always one more than the actual merge
		adj_sc = start_col
		adj_ec = end_col - 1

		# Make sure to avoid out-of-bounds errors
		adj_sr = min(adj_sr, max_rows - 1)
		adj_sc = min(adj_sc, max_cols - 1)
		adj_er = min(adj_er, max_rows - 1)
		adj_ec = min(adj_ec, max_cols - 1)

		# Extract the starting values
		x_tok_start = x_tok[ adj_sr, adj_sc, : ].clone()
		y_tok_start = y_tok[ adj_sr, adj_sc, : ].clone()
		#check_df_start = check_df.iloc[adj_sr, adj_sc]

		# print("\nBefore Propagation:")
		# print("check_df:")
		# for row in range(adj_sr, adj_er + 1):
		#     for col in range(adj_sc, adj_ec + 1):
		#         print(f"\t({row}, {col}): {check_df.iloc[row, col]}")
		# 
		# print("\nx_tok:")
		# for row in range(adj_sr, adj_er + 1):
		#     for col in range(adj_sc, adj_ec + 1):
		#         print(f"\t({row}, {col}):\n\t{x_tok[row, col, :]}")
		# 
		# print("\ny_tok:")
		# for row in range(adj_sr, adj_er + 1):
		#     for col in range(adj_sc, adj_ec + 1):
		#         print(f"\t({row}, {col}):\n\t{y_tok[row, col, :]}")

		# Propagate values across the merged range
		for row, col in itertools.product(range(adj_sr, adj_er + 1), range(adj_sc, adj_ec + 1)):
			x_tok[ row, col, : ] = x_tok_start
			#check_df.iloc[row, col] = check_df_start

			# Duplicate y_tok values excluding indices 15 and 16
			y_tok[ row, col, :15 ] = y_tok_start[ :15 ]
			y_tok[ row, col, 17: ] = y_tok_start[ 17: ]

		# # Print values after propagation  # print("\nAfter Propagation:")  # print("check_df:")  # for row in range(adj_sr, adj_er + 1):  #     for col in range(adj_sc, adj_ec + 1):  #         print(f"\t({row}, {col}): {check_df.iloc[row, col]}")  #   # print("\nx_tok:")  # for row in range(adj_sr, adj_er + 1):  #     for col in range(adj_sc, adj_ec + 1):  #         print(f"\t({row}, {col}):\n\t{x_tok[row, col, :]}")  #   # print("\ny_tok:")  # for row in range(adj_sr, adj_er + 1):  #     for col in range(adj_sc, adj_ec + 1):  #         print(f"\t({row}, {col}):\n\t{y_tok[row, col, :]}")

	# Return the 100x100 2D NumPy array containing the metadata for each cell
	return x_tok, y_tok

In [8]:
# Import self funcs
import importlib
from utils import selfutil
importlib.reload(selfutil)
from utils.selfutil import tokenize_pad

# Import parsing libraries
import xlrd
from xlrd.xldate import xldate_as_tuple, XLDateAmbiguous, XLDateError
import openpyxl
import csv
import re
import sys
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

# Import general usecase libraries
import pandas as pd
import torch
from datetime import datetime, time
import warnings
import itertools
from typing import Optional, Tuple

test_file = '../data/val_big/sara_shackleton_001_1_1.pst.346.xls'
#test_file = '../data/train_big/parse_input.xls'

test_df, y_tok = process_xls(test_file, spreadsheet_vocab, 100, 100, 32)
display(test_df)

ValueError: 70: ../data/val_big/sara_shackleton_001_1_1.pst.346.xls

In [24]:
import pandas as pd

# Extract dimensions from y_tok for safety
max_rows, max_cols, _ = y_tok.shape

# Initialize a DataFrame to store the combined values
combined_df = pd.DataFrame(index=range(max_rows), columns=range(max_cols))

# Iterate over every cell in y_tok
for row in range(max_rows):
    for col in range(max_cols):
        # Extract the 15th and 16th indices for the current cell
        value1 = y_tok[row, col, 15].item()  # Convert to Python native data type
        value2 = y_tok[row, col, 16].item()
        
        # Combine the two values as a string separated by a comma
        combined_value = f"{value1},{value2}"
        
        # Store the combined value in the DataFrame
        combined_df.iloc[row, col] = combined_value

# Display the resulting DataFrame
display(combined_df)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
1,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
2,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
3,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
4,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
5,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
6,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"

## XLSX Files

### Original process_xlsx function


In [92]:
#test_file = '../data/train_big/michelle_lokay_000_1_2_1.pst.149.xls'
test_file = '../data/train_big/parse_input.xlsx'

test_df, y_tok = process_xlsx(test_file, spreadsheet_vocab, 100, 100, 32)
display(test_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,,Monday,Tuesday,Wednesday,Thursday,friday,saturday,Sunday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,Val1,test,2024-09-08 00:00:00,1pm,Boolean,dh,Some,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2,20,40,80,160,320,640,1280,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,3,20,40,80,36.285031392744614,45.85725806142466,This is a multiple word sentence,1816.806957563503,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,4,20,40,80,42.22922635061216,43.519774554408265,Test,TestCool,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,5,20,40,80,42.03629416295762,43.11781037113451,Test2,Test232.051254611368,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,6,20,1.23e-09,2.46e-09,41.8258226855163,42.57886290822945,Test3,Test330.6894164120177,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,7,20,1.23e+02,246,41.59530630546152,41.98748867653515,Test,Test29.2549552412625,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,8,20,1.23e+02,246,43.34173828733976,43.517245828178766,Test,Test27.624319875906,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,9,20,40,80,38.798321004217286,38.1317925816225,1999-01-02 00:00:00,3616225.7523591579387,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## NEW test_xlsx function

In [149]:
# Import self funcs
import importlib
from utils import selfutil
importlib.reload(selfutil)
from utils.selfutil import tokenize_pad

# Import parsing libraries
import xlrd
from xlrd.xldate import xldate_as_tuple, XLDateAmbiguous, XLDateError
import openpyxl
import csv
import re
import sys
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

# Import general usecase libraries
import pandas as pd
import torch
from datetime import datetime, time
import warnings
import itertools
from typing import Optional, Tuple

def test_xlsx(file_path, vocab, max_rows, max_cols, pad_length):

    def get_dataType(value, number_format):
        """
        Determines the type of the cell and handles necessary conversions.

        Int to Type Key Mapping:
            0  -> Empty cell
            1  -> Text cell
            2  -> Numeric type
            3  -> Integer subclass of number
            4  -> Float subclass of number
            5  -> Percentage subclass of number
            6  -> Currency subclass of number
            7  -> Scientific subclass of number
            8  -> Date type
            9  -> Time subclass of date
            10 -> Datetime subclass of date
            11 -> Boolean type with T or F value
            12 -> Error type corresponding to #REF!, #VALUE! etc. in excel
            13 -> Blank cell with formatting
            14 -> Unknown type not in our keys

        Args:
            value: The value of the cell.
            number_format (str): The number format of the cell.

        Returns:
            int: The determined type of the cell (0-14).
        """
        # Empty cell with no value return blank for this
        if value is None:
            return 13

        # Boolean cell
        elif isinstance(value, bool):
            return 11

        # Error cell based on certain error values in Excel
        elif any(error in str(value) for error in {"#VALUE!", "#REF!", "#DIV/0!", "#NAME?", "#N/A", "#NUM!", "#NULL!"}):
            return 12

        # Currency format cell
        elif "#,##0" in number_format:
            return 6

        # Percentage format cell
        elif number_format.endswith("%"):
            return 5

        # Check for scientific notation format
        elif "E+" in number_format or "E-" in number_format:
            return 7

        # If float type
        elif isinstance(value, float):
            # Check if it's an integer subclass
            if value.is_integer():
                return 3
            # Otherwise, it's a float
            return 4

        # If integer type
        elif isinstance(value, int):
            return 3

        # Time type cell
        elif isinstance(value, time):
            return 9

        # Datetime type cell
        elif isinstance(value, datetime):
            # Check if the time part is exactly midnight
            if value.time() == time(0, 0):
                # If the datetime is formatted as a date, return it as a date type
                if "d" in number_format.lower() and "h" not in number_format.lower():
                    return 8
                else:
                    # If there is significant time information, return it as datetime
                    return 10
            else:
                # If the time part is not midnight, it's definitely a datetime
                return 10

        # Blank cell with formatting
        elif value == '':
            return 13

        # Text cell
        elif isinstance(value, str):
            return 1

        # Return 14 for any unknown type
        return 14

    def get_value(cell_type, cell):
        # If empty cell then return an empty string
        if cell_type == 0:
            return ''

        # Text cell
        elif cell_type == 1:
            return str(cell.value)

        # Numeric cell default type
        elif cell_type == 2:
            return str(cell.value)

        # Integer subclass of number, remove proceeding 0s and return str
        elif cell_type == 3:
            return str(cell.value)

        # Decimal subclass then convert to float and return str
        elif cell_type == 4:
            return str(cell.value)

        # Percentage subclass then convert to value add % sign and return the string
        elif cell_type == 5:
            return str(cell.value * 100) + '%'

        # Currency subclass then convert to value add currency symbol and return the string
        elif cell_type == 6:
            # Check if cell value already is a string so it will have the symbol
            if isinstance(cell.value, str):
                return str(cell.value)
            # Check if euro sign in format string because that is separate
            if '$€' in cell.number_format:
                return '€' + str(float(cell.value))
            # In other case for usd values we gotta add the $ symbol
            return '$' + str(float(cell.value))

        # If scientific subclass then convert to value add 'E' sign and return the string
        elif cell_type == 7:
            if isinstance(cell.value, (int, float)):
                return f"{cell.value:.2e}"
            else:
                return str(cell.value)

        # If date subclass then use the df to return value
        elif cell_type in [8, 9, 10]:
            return str(cell.value)

        # If boolean cell then use df to get string
        elif cell_type == 11:
            return str(cell.value)

        # If error cell then use df to get string
        elif cell_type == 12:
            return str(cell.value)

        # If blank cell (Empty cell with formatting) then return an empty string
        elif cell_type == 13:
            return ''

        # Return empty string for default case
        return ''

    def get_alignment(cell):
        # Define dict to store the alignment keys mapped to xlrd
        openpyxl_to_xlrd = {
            'horizontal': {'general': 0, 'left': 1, 'center': 2, 'right': 3, 'fill': 4, 'justify': 5, 'centerContinuous': 6, 'distributed': 7},
            'vertical': {'top': 0, 'center': 1, 'bottom': 2, 'justify': 3, 'distributed': 4},
        }

        # Get the horizontal and vertical alignment else default if key not found
        horiz = openpyxl_to_xlrd['horizontal'].get(cell.alignment.horizontal, 0)
        vert = openpyxl_to_xlrd['vertical'].get(cell.alignment.vertical, 2)

        # Concatenate the keys and return as int
        return horiz, vert

    def get_borders(cell):
        # Define dict to store the border keys mapped to xlrd
        openpyxl_to_xlrd = {
            'none': 0, 'thin': 1, 'medium': 2, 'dashed': 3, 'dotted': 4, 'thick': 5, 'double': 6, 'hair': 7, 'mediumDashed': 8, 'dashDot': 9, 'mediumDashDot': 10, 'dashDotDot': 11,
            'mediumDashDotDot': 12, 'slantDashDot': 13
        }

        return openpyxl_to_xlrd.get(cell.border.top.style or 'none'), openpyxl_to_xlrd.get(cell.border.bottom.style or 'none'), openpyxl_to_xlrd.get(cell.border.left.style or 'none'), openpyxl_to_xlrd.get(cell.border.right.style or 'none')

    def get_merge(row, col, sheet):
        """
        Determines the merge status of a cell in terms of its vertical and horizontal position within a merged range.

        Int to Merge Status Mapping:
            1 -> Outside any merged range ('O')
            2 -> Beginning of a merged range ('B')
            3 -> Inside a merged range but not at the beginning ('I')

        Args:
            row (int): The row index of the cell.
            col (int): The column index of the cell.
            sheet (openpyxl.worksheet.worksheet.Worksheet): The sheet object containing the merged cells information.

        Returns:
            int: A 2-digit integer where the first digit represents the vertical position
                 and the second digit represents the horizontal position within a merged range.
        """
        # Initialize tags as 1 (outside) by default
        vertical_tag = 0
        horizontal_tag = 0

        # Convert 0-based indexing to 1-based indexing for openpyxl
        row_1_based = row + 1
        col_1_based = col + 1
        cell_coord = openpyxl.utils.get_column_letter(col_1_based) + str(row_1_based)

        # Iterate over the merged cells in the sheet
        for merged in sheet.merged_cells.ranges:
            # Check if the current cell is within a merged range
            if cell_coord in merged:
                # Determine vertical tag
                if row_1_based == merged.min_row:
                    vertical_tag = 1  # Beginning of vertical range
                else:
                    vertical_tag = 2  # Inside vertical range

                # Determine horizontal tag
                if col_1_based == merged.min_col:
                    horizontal_tag = 1  # Beginning of horizontal range
                else:
                    horizontal_tag = 2  # Inside horizontal range

                # Break the loop as the cell is part of a merged range
                break

        # Return the combined vertical and horizontal tags as a 2-digit integer
        return horizontal_tag, vertical_tag

    def get_fontcol(cell):
        # Check if None color
        if cell.font.color is None:
            return 0

        # Check tint first of all
        if float(cell.font.color.tint) != 0.0:
            return 1

        # Check theme
        if cell.font.color.theme is not None and cell.font.color.theme != 1:
            return 1

        # Check if rgb exists and is not None
        if cell.font.color.rgb is not None:
            # Ensure the RGB value is 8 characters long (including opacity) and starts with 'FF' for full opacity
            if isinstance(cell.font.color.rgb, str) and len(cell.font.color.rgb) == 8:
                # Extract the last 6 characters which represent the color and check if it's not black
                font_color = f"#{cell.font.color.rgb[2:].lower()}"

                # If color is not black return 1 else 0
                if font_color != '#000000':
                    return 1  # Font is colored
                else:
                    return 0  # Font is black (default)
            else:
                return 0  # RGB value doesn't match expected format

        # If rgb is None
        else:
            return 0

    # Suppress all UserWarnings related to various xlsx issues
    warnings.filterwarnings("ignore", category=UserWarning)

    # Initialize 3D tensor of size rows x cols x pad_length = 100x100x32
    x_tok = torch.zeros((max_rows, max_cols, pad_length), dtype=torch.long)

    # Initialize y_tok tensor to store metadata for each cell of size row x col x 6
    y_tok = torch.zeros((max_rows, max_cols, 17), dtype=torch.long)
    
    # Initialize checking df DataFrame to store cell values for parse test
    #check_df = pd.DataFrame('', index=range(max_rows), columns=range(max_cols))

    # Load the workbook and access the active sheet
    workbook = openpyxl.load_workbook(file_path, data_only=True)

    # Retrieve the sheet
    sheet = workbook.active

    # Iterate over each combination of rows and columns in the sheet, up to the 100th row and column
    for row, col in itertools.product(range(min(sheet.max_row, max_rows)), range(min(sheet.max_column, max_cols))):
        # Adjust to 1-based indexing
        cell = sheet.cell(row=row + 1, column=col + 1)

        # Determine the cell type
        cell_type = get_dataType(cell.value, cell.number_format)

        # Determine the value of the cell
        cell_value = get_value(cell_type, cell)

        # Adjusted logic for determining if the cell is filled
        is_fill = 0 if (cell.fill.start_color.index if cell.fill and cell.fill.start_color else None) in ['00000000', '00FFFFFF', 0] else 1

        # Get alignment of the cell
        halign, valign = get_alignment(cell)

        # Determine font family if None then default to 0
        cell_fam = int(cell.font.family) if cell.font.family else 0

        # Determine font size
        cell_size = cell.font.size

        # Determine cell style
        is_bold = int(cell.font.bold)
        is_italic = int(cell.font.italic)
        cell_underline = 1 if cell.font.underline == 'single' else 2 if cell.font.underline == 'double' else 0
        cell_scr = 1 if cell.font.vertAlign == 'superscript' else 2 if cell.font.vertAlign == 'subscript' else 0

        # is_col = 0 if font_color == '#000000' else 1
        is_col = get_fontcol(cell)

        # Determine borders
        btop, bbot, bleft, bright = get_borders(cell)

        # Get merged range
        hmerge, vmerge = get_merge(row, col, sheet)

        # Set the tokens for current cell in x_tok
        x_tok[row, col, :] = torch.tensor(tokenize_pad(cell_value, vocab, pad_length), dtype=torch.long)
        #check_df.iloc[row, col] = cell_value
        
        # Set y_tok for current cell in y_tok
        y_tok[row, col, 0] = cell_type
        y_tok[row, col, 1] = is_fill  # Store 'fill'
        y_tok[row, col, 2] = halign  # Store 'halign'
        y_tok[row, col, 3] = valign  # Store 'valign'
        y_tok[row, col, 4] = cell_fam  # Store 'font family'
        y_tok[row, col, 5] = cell.font.size  # Store 'font size'
        y_tok[row, col, 6] = is_bold  # Store 'bold'
        y_tok[row, col, 7] = is_italic  # Store 'italic'
        y_tok[row, col, 8] = cell_underline  # Store 'underline'
        y_tok[row, col, 9] = cell_scr  # Store 'escapement'
        y_tok[row, col, 10] = is_col  # Store 'colour'
        y_tok[row, col, 11] = btop  # Store top border style
        y_tok[row, col, 12] = bbot  # Store bottom border style
        y_tok[row, col, 13] = bleft  # Store 'left border style'
        y_tok[row, col, 14] = bright  # Store 'right border style'
        y_tok[row, col, 15] = hmerge
        y_tok[row, col, 16] = vmerge  # Store 'hmerge' and 'vmerge'
        
            
        # If it is an empty/blank cell then further processing is required
        if y_tok[row, col, 0] in [0, 13, 14]:
            
            # Set font-related metadata to 0 (indices 4 to 10)
            y_tok[row, col, 4:11] = 0
        
            # If the cell is further of the type center across cells
            if y_tok[row, col, 2 ] == 6:
                
                # Define variable to store col of where this center continuous is starting
                start_col = col

                # Loop until we find a cell that is centered across cells but not empty or blank
                while y_tok[row, start_col, 2 ] == 6 and ( y_tok[row, start_col, 0 ] == ( 0 or 13 ) ):
    
                    # Move to the left
                    start_col = start_col - 1
                
                # Set the x_tok and y_tok tensor to these values
                #check_df.iloc[row, col] = check_df.iloc[row, start_col]
                x_tok[row, col, :] = x_tok[row, start_col, :]
                y_tok[row, col, :] = y_tok[row, start_col, :]
                
    
    # Loop through merged cell ranges
    for merged in sheet.merged_cells.ranges:
        # Parse the range into start and end coordinates
        start_cell, end_cell = str(merged).split(':')
        
        # Convert start cell to row and column
        start_col, start_row = coordinate_from_string(start_cell)
        start_row = min(start_row - 1, max_rows - 1)  # Convert to 0-based indexing
        start_col = min(column_index_from_string(start_col) - 1, max_cols - 1)  # Convert to 0-based indexing
        
        # Convert end cell to row and column
        end_col, end_row = coordinate_from_string(end_cell)
        end_row = min(end_row, max_rows - 1)  # Convert to 0-based indexing
        end_col = min(column_index_from_string(end_col), max_cols - 1)  # Convert to 0-based indexing
        
        # Extract the starting values
        x_tok_start = x_tok[start_row, start_col, :].clone()  # Clone for safety
        #check_df_start = check_df.iloc[start_row, start_col]
        y_tok_start = y_tok[start_row, start_col, :].clone()
        
        # Propagate values across the merged range
        for row, col in itertools.product(range(start_row, end_row), range(start_col, end_col)):
            x_tok[row, col, :] = x_tok_start
            #check_df.iloc[row, col] = check_df_start

            # Duplicate y_tok values excluding indices 15 and 16
            y_tok[row, col, :15] = y_tok_start[:15]
            y_tok[row, col, 17:] = y_tok_start[17:]


    # Return the 2D NumPy array containing the metadata for each cell
    return x_tok, y_tok

In [150]:
#test_file = '../data/train_big/michelle_lokay_000_1_2_1.pst.149.xls'
test_file = '../data/train_big/gifted-talented-enrollment.xlsx'

test_df, y_tok = test_xlsx(test_file, spreadsheet_vocab, 100, 100, 32)
display(test_df)

tensor([[[  2,   3,   1,  ...,   1,   1,   1],
         [  2,   3,   1,  ...,   1,   1,   1],
         [  2,   3,   1,  ...,   1,   1,   1],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[  2,   3,   1,  ...,   1,   1,   1],
         [  2,  90,   4,  ...,  10, 800,  12],
         [  2,  90,   4,  ...,  10, 800,  12],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[  2,   3,   1,  ...,   1,   1,   1],
         [  2,   3,   1,  ...,   1,   1,   1],
         [  2,   3,   1,  ...,   1,   1,   1],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]

In [142]:
import os
from tqdm import tqdm  # For progress bar
import pandas as pd

# Define the directory containing the .xlsx files
data_dir = '../data/train_big/'

# Extract all .xlsx file paths into a list
xlsx_files = [os.path.join(data_dir, file_name) for file_name in os.listdir(data_dir) if file_name.endswith('.xlsx')]

# Initialize a list to track files that failed processing
failed_files = []

# Iterate through the list of .xlsx files with a progress bar
for file_path in tqdm(xlsx_files, desc="Processing .xlsx files"):
    try:
        # Attempt to process the file
        test_df, y_tok = test_xlsx(file_path, spreadsheet_vocab, 100, 100, 32)
    except Exception as e:
        # Log the file name and the error
        print(f"{file_path}: {e}")
        failed_files.append(file_path)

# Print summary of failed files
print("\nProcessing completed.")
if failed_files:
    print("The following files failed processing:")
    for failed_file in failed_files:
        print(failed_file)
else:
    print("All files processed successfully.")


Processing .xlsx files: 100%|██████████| 389/389 [03:31<00:00,  1.84it/s]


Processing completed.
All files processed successfully.
